In [1]:
# Obtiene la curva de distribución acumulada del modelo.

import os

import numpy as np
import pandas as pd

import xarray as xr

In [2]:
v = "Pcp"
dims = [ "XTIME", "XLAT", "XLONG" ]

# 2 futuro
# 1 actual
# 0 observado
model = 2

if model == 2:
    path_d = "../temp/WRF_2040_regrid_CHIRPS_days.nc"
    path_r = "../temp/WRF_2040_regrid_CHIRPS_days_cdf.nc"
elif model == 1:
    path_d = "../temp/WRF_regrid_CHIRPS_days.nc"
    path_r = "../temp/WRF_regrid_CHIRPS_days_cdf.nc"
else:
    path_d = "../data/CHIRPS_megalopolis.nc"
    path_r = "../temp/CHIRPS_megalopolis_cdf.nc"

with xr.open_dataset(path_d) as ds:
    if not model:
        ds = ds.rename( {"time": "XTIME", "longitude": "XLONG", 
            "latitude": "XLAT", "precip": "Pcp"})

    ds = ds[ ["XTIME", "XLAT", "XLONG", "Pcp"] ]
    ds_2 = ds.drop_vars( list(ds.keys()) ).copy() 
    df = ds.to_dataframe().sort_index()

    latitude = df.index.get_level_values(dims[1]).unique()
    longitude = df.index.get_level_values(dims[2]).unique()

    # Obtenemos la curva de distribución acumulada para cada celda.
    df["q_model"] = 0.0
    for lat in latitude:
        for lon in longitude:
            df_c = df.loc[ (slice(None), lat, lon),
                [v, "q_model"] ].sort_values(v)
            df_c["q_model"] = np.linspace(
                1/df_c.shape[0], 1, df_c.shape[0] )
            df.loc[ (slice(None), lat, lon), "q_model"
                ] = df_c["q_model"]

    ds_2["q_model"] = df["q_model"].to_xarray()
    ds["q_model"] = ( dims, ds_2["q_model"].values )
    # Guardamos el archivo.
    ds.to_netcdf( path_r, mode = "w" )